Лабораторная работа 2. Обработка текста.

Задание:

Написать программу на языке программирования Python, реализующую:
1)	Предварительную обработку текста, включающую:
	-	Перевод текста в нижний регистр
	-	Удаление знаков препинания
	-	Удаление стоп-слов
	-	Удаление лишних символов
	-	Стемминг и лемматизацию
2)	Преобразование текста в мешок слов
3)	Преобразование текста в N-граммы, где N=2,4

Замечание!
Желательно провести предварительную обработку текста с использованием библиотеки NLTK.

In [17]:
import nltk 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem.snowball import SnowballStemmer # Стеммер для русского языка
from nltk.stem.porter import PorterStemmer # Стеммер для английского 
from nltk.stem import WordNetLemmatizer # Лемматайзер для английского
from nltk.corpus import wordnet
from pymystem3 import Mystem # Лемматайзер для русского языка 
import re
import string
from langdetect import detect
import logging
import os 
from datetime import datetime
from sklearn.feature_extraction.text import CountVectorizer
from spellchecker import SpellChecker
from num2words import num2words
import requests
import enchant
from enchant.checker import SpellChecker as EnchantChecker 


In [18]:
# Конфигурационный блок 

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vadig\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vadig\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vadig\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\vadig\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\vadig\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [19]:
log_dir = r'D:\Projects\SvetlanaDmitrievna\lab3_ii\logs'
os.makedirs(log_dir, exist_ok=True) 
log_filename = os.path.join(log_dir, f'{datetime.now().strftime("%H %M %S %Y %m %d")} nlp.log')

# Удаляем логеры, которые уже запущены по случайности 
logger = logging.getLogger()
for handler in logger.handlers[:]:
    logger.removeHandler(handler)
    
# Настройки для логгера 
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_filename),
        logging.StreamHandler()  # Вывод в консоль 
    ]
)
logger = logging.getLogger(__name__)


In [20]:
# Для работы с txt 

# Пути к файлам 
english_output_file_path = 'D:/Projects/SvetlanaDmitrievna/lab3_ii/texts/output_english.txt'
russian_output_file_path = 'D:/Projects/SvetlanaDmitrievna/lab3_ii/texts/output_russian.txt'

def read_txt(file_path): 
	"""
	Считывает текст из файла. 

	:param file_path: путь к файлу 
	:return: содержимое файла в виде строки 
	"""

	logger.info(f'Reading text from file: {file_path}')

	try: 
		with open(file_path, 'r', encoding='utf-8') as file: 
			text = file.read()

		logger.info(f'Successfully read {len(text)} chars from file.')

		return text
	
	except Exception as e: 
		logger.critical(f'Didn`t get a chance to read the text: {str(e)}')
		return ""
	
	
def text_to_txt(file_path, text, new_text=True, step='Start. To lower case.'): 
	"""
	Записывает текст в txt. 

	:param file_path: путь к файлу, куда записать текст
	:param text: сам обработанный текст
	:param new_text: Если True, то стираются прошлые данные в файле, по сути создаётся новый файл. Если False, то дополняется новым текстом. 
	:param step: на каком этапе генерация файла. Например, 'Lemmatization' и т.д.
	""" 
	
	if step == 'Final Preproccessed Text':
		logger.info(f'Transmitted preprocessed text with {len(text)} chars.')

	mode = 'w' if new_text else 'a'

	try: 
		with open(file_path, mode, encoding='utf-8') as file: 
			
			start_line = f'[{step} — {datetime.now().strftime("%H %M %S %Y %m %d")}]\n\n' if new_text else f'\n\n\n[{step} — {datetime.now().strftime("%H %M %S %Y %m %d")}]\n\n'

			# Этот блок кода сделан специально, чтобы текст можно было читать не одним
			# сплошным предложением
			text = text.split()
			line = ''
			result = [start_line, ]
			for i, word in enumerate(text): 
				if i % 15 == 0 and i != 0: 
					line.rstrip()
					line += '\n'
					result.append(line)
					line = f'{word} '
				else: 
					line += f'{word} '

			if line: result.append(line)

			result = ''.join(result)
		
			file.write(result)

			logger.info(f'The text for step — {step} — is written to {file_path}')

	except Exception as e: 
		logger.critical(f'Error to write the text: {str(e)}')

In [ ]:
class TextPreprocessing: 
	"""
	Делает упорядоченную предобработку текста: 
	1. Перевод текста в нижний регистр. 
	2. Исправление опечаток. 
	3. Числа в строки.
	4. Удаление пунтуации и других символов.  
	5. Удаление стоп-слов. 
	6. Нормализация текста. Лемматизация / стемминг. 
	"""

	def __init__(self, text, delete_punctuation_mode='string', text_normalization_method='lemma'):
		"""
		Инициализация класса TextPreprocessing.

		:param text: исходный текст для предобработки
		"""

		logger.info('Text is initializing...')

		self.text = text
		self.delete_punctuation_mode = delete_punctuation_mode
		self.language = self.__detect_language() 
		self.normalization_method = text_normalization_method

		logger.info(f'Text is initialized with parameters: delete_punctuation_mode={delete_punctuation_mode}, text_normalization_method={text_normalization_method}')
 
	def __detect_language(self) -> str: 
		"""
		Автоматическое определение языка с помощью библиотеки langdetect. 

		:return: Возвращается формат языка в стиле 'russian', 'english'.
		"""

		try:
			detected_lang = detect(self.text)
			
			lang_mapping = {
				'ru': 'russian',
				'en': 'english',
				# TODO: create more languages
			}

			language = lang_mapping[detected_lang]

			logger.info(f'Detected language: {language}')
			
			return language
		
		except:
			logger.critical('Error with detecting langauge.')

	def __write_to_file(self, result_text, new_text=True, step='Start. To lower case.'): 
		"""
		Записывает в файл текст на каком-то этапе, использую функцию text_to_txt()

		:param result_text: обработанный на каком-то этапе текст
		:param new_text: Если True, то стираются прошлые данные в файле, по сути создаётся новый файл. Если False, то дополняется новым текстом.
		:param step: на каком этапе генерация файла. Например, 'Lemmatization' и т.д.
		"""

		match self.language: 
			case 'russian':
				text_to_txt(russian_output_file_path, result_text, new_text=new_text, step=step)
			case 'english': 
				text_to_txt(english_output_file_path, result_text, new_text=new_text, step=step)
			case _:
				logger.warning(f'No method has been developed to allow writing to a file on {self.language}')


	def __lang_to_simple_form(self) -> str: 
		"""
		Вовзвращает упрощенную форму языка. 

		'english' -> 'en' 
		"""

		lang_mapping = {
				'russian': 'ru', 
				'english': 'en'
			}
		
		return lang_mapping[self.language]
 
	def __to_lower_case(self) -> str:
		"""
		Приводит текст к нижнему регистру. 

		:return: возвращаем измененный текст. 
		"""
		
		logger.info('Text has been converted to lower case.')

		result_text = self.text.lower() 

		self.__write_to_file(result_text, new_text=True, step='Start. To lower case.')

		return result_text
	
	def __correct_spell(self) -> str: 
		"""
		Исправляет опечатки в тексте. 
		
		:param text: исходный текст.
    :return: текст с исправленными опечатками.
		"""

		logger.info('Correction of typos initiated...')

		try: 

			if self.language == 'english':
				spell = SpellChecker(language=self.__lang_to_simple_form()) 
				words = self.text.split() 
				corrected_words = [] 
				for word in words: 

					if word.isdigit() or len(word) <= 2: 
						corrected_words.append(word)
						continue

					corrected_word = spell.correction(word)
					corrected_words.append(corrected_word if corrected_word else word)

				logger.info(f'Correction of typos initiated done with {len(corrected_words)} corrected words.')
				result_text = ' '.join(corrected_words)

				self.__write_to_file(result_text, new_text=False, step='Spell correction.')

				return result_text
			
			# Для русского языка используем Яндекс.Спеллер (не работает на данный момент)
			elif self.language == 'russian':
					
					url = "https://speller.yandex.net/services/spellservice.json/checkText"
					params = {
							'text': self.text,
							'lang': 'ru',
							'format': 'plain'
					}
					
					response = requests.get(url, params=params)
					
					if response.status_code == 200:
							corrections = response.json()
							result_text = self.text
							
							# Применяем исправления в обратном порядке (с конца текста)
							for item in reversed(corrections):
									pos = item['pos']
									length = item['len']
									suggestions = item['s']
									
									if suggestions:
											# Заменяем ошибочное слово на первое предложенное исправление
											result_text = result_text[:pos] + suggestions[0] + result_text[pos + length:]
							
							logger.info(f'Corrected {len(corrections)} typos using Yandex.Speller')

							self.__write_to_file(result_text, new_text=False, step='Spell correction.')

							return result_text
					else:
						logger.error(f'Yandex.Speller API returned status code {response.status_code}')
						return self.text

		except Exception as e: 
			logger.error(f'Typos correction error: {str(e)}')
			return self.text
	
	def __numbers_to_words(self) -> str: 
		"""
    Заменяет числа в тексте на их текстовые эквиваленты.
    
    Например:
    - "10 книг" -> "десять книг"
    - "2 students" -> "two students"
    
    :return: текст с числами, преобразованными в слова
    """
		
		logger.info('Converting numbers to words...')
		try: 

			num_lang = self.__lang_to_simple_form()

			def replace_number(match): 
				'''
				Вызывается, когда находится соответствие маске '\b\d+[,.]?\d*\b'

				:param match: это часть текста, которая маске подошла.
				'''
				try: 
					# .group(0) - возвращает весь текст, а не только его какую-то группу. 
					number = match.group(0)

					if '.' in number or ',' in number:
						number = number.replace(',', '.')
						return num2words(float(number), lang=num_lang)
					else: 
						return num2words(int(number), lang=num_lang)
					
				except ValueError: 
					
					return match.group(0)
					
			result_text = re.sub(r'\b\d+[,.]?\d*\b', replace_number, self.text)

			logger.info('Numbers have been converted to words.')

			self.__write_to_file(result_text, new_text=False, step='Numbers to string.')

			return result_text
		
		except Exception as e: 
			logger.error(f'Error during number to word conversion: {str(e)}')
			return self.text

	def __delete_punctuation_marks(self) -> str:
		"""
		Удаляет знаки препинания из предложения.

		Режимы (delete_punctuation_mode): 
		1. 're' — c помощью регулярных выражений. 
		2. 'string' — с помощью библиотеки string.  

		:return: текст без знаков препинания. 
		"""

		logger.info('Punctuation are being deleting...')

		match self.delete_punctuation_mode: 
			case 're':
				result_text = re.sub(r'[^\w\s]', '', self.text)
			
			case 'string':
				# Параметры: 
				# Первый аргумент '' символы для замены 
				# Второй аргумент '' символы на которые заменяем 
				# Третий аргумент string.punctuation символы, которые нужно удалить 
				# string.punctuation = "!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~" 
				translator = str.maketrans('', '', ''.join(c for c in string.punctuation if c != ' '))

				# translate() применяет таблицу преобразования к строке 
				result_text = self.text.translate(translator)
			
		logger.info('Punctuation has been deleted.')

		self.__write_to_file(result_text, new_text=False, step='Delete punctuation.')

		return result_text 

	def __delete_stop_words(self) -> str: 
		"""
		Удаляет стоп-слова. 

		Стоп-слова — распространенные слова в языке, которые не несут значимой смысловой нагрузки 
		при анализе текста. 

		Зачем их удалять? 
		1. Текст становится меньше. 
		2. Фокус на важных словах. 
		"""

		logger.info('Stop words are being removed...')

		stopwords.words(self.language)

		stop_words = set(stopwords.words(self.language))
		
		words = self.text.split()

		filtered_words = [word for word in words if word not in stop_words]

		result_text = ' '.join(filtered_words)

		logger.info('Stop words have been removed.')

		self.__write_to_file(result_text, new_text=False, step='Delete stop words.')
		
		return result_text 
		
	def __stemming(self) -> str: 
		"""
		Выполняет стемминг слов в тексте.
		
		Стемминг — процесс нахождения основы слова (стеммы) путем 
		отбрасывания окончаний и суффиксов. Например, для слов "рыбак", 
		"рыбачить", "рыбный" стеммой будет "рыб".
		
		:return: текст со словами, приведенными к их основам. 
		"""

		logger.info('Stemming is being performed...')
		
		try: 
			
			words = self.text.split() 
			stemmed_words = [] 

			match self.language: 
				case 'russian': 
					stemmer = SnowballStemmer("russian")
				case 'english': 
					stemmer = PorterStemmer()
				case _: 
					logger.error(f'Stemming not supported for language: {self.language}')
					return self.text

			for word in words:
				stemmed_words.append(stemmer.stem(word))
			
			result_text = ' '.join(stemmed_words)

			logger.info('Stemming has been performed.')

			self.__write_to_file(result_text, new_text=False, step='Stemming.')

			return result_text
			
		except Exception as e: 
			logger.error(f'Error during stemming: {str(e)}')
			return self.text
		
	def __lemming(self) -> str: 
		"""
		Выполняет лемматизацию слов в тексте.
			
		Лемматизация — процесс приведения слова к его словарной форме (лемме).
		Например, для слов "бегущий", "бегает", "бежит" леммой будет "бежать".
			
		:return: текст со словами в словарной форме
		"""

		logger.info('Lemmatizaion is being performed...')

		try: 
			
			words = self.text.split() 
			lemmatized_words = []

			match self.language: 
				case 'russian': 
						lemmatizer = Mystem()

						for word in words: 
							try: 
								# .parse(word) — библиотека pymorphy2 анализирует слово и 
								# возвращает список всех возможных морфологических разборов этого слова.
								# мы берем [0] — наиболее вероятный
								#
								# После чего делаем нормальную форму (лемму) .normal_form
								# Для существительных - именительный падеж, единственное число
								# Для глаголов - инфинитив
								# Для прилагательных - мужской род, единственное число, именительный падеж
								lemma = ''.join(lemmatizer.lemmatize(word)).strip()
								lemmatized_words.append(lemma)
							
							except Exception as e:
								logger.warning(f"Failed to lemmatize word '{word}': {e}")
								lemmatized_words.append(word)  # Сохраняем исходное слово

				case 'english':

					lemmatizer = WordNetLemmatizer()

					# Разобраться, что делает эта функция 
					def get_wordnet_pos(word):
						"""Отображает тег части речи NLTK на тег WordNet"""
						tag = nltk.pos_tag([word])[0][1][0].upper()
						tag_dict = {
								'J': wordnet.ADJ,
								'N': wordnet.NOUN,
								'V': wordnet.VERB,
								'R': wordnet.ADV
						}
						return tag_dict.get(tag, wordnet.NOUN)
					
					for word in words: 
						lemma = lemmatizer.lemmatize(word, get_wordnet_pos(word))
						lemmatized_words.append(lemma)

				case _: 
					logger.error(f'Lemmatization not supported for language: {self.language}')
					return self.text
			
			result_text = ' '.join(lemmatized_words)

			logger.info('Lemmatization has been performed.')

			self.__write_to_file(result_text, new_text=False, step='Lemmatization.')

			return result_text
			
		except Exception as e: 
			logger.error(f'Error during lemmatizaion: {str(e)}')
			return self.text

	def __text_normalization(self) -> str: 
		"""
		Выбирает метод нормализации текста. 
		"""

		match self.normalization_method: 
			case 'lemma': 
				return self.__lemming()
			case 'stem': 
				return self.__stemming()
			case _: 
				logger.warning('There is no such thing as a text normalization method')
				logger.info('Returned text without normalization')
				return self.text

	def process_text(self) -> str:
		"""
		Обрабатывает текст, применяя все методы предобработки.
		
		:return: обработанный текст по всем функциям.
		""" 

		for func in [self.__to_lower_case, self.__correct_spell, self.__numbers_to_words, self.__delete_punctuation_marks, self.__delete_stop_words, self.__text_normalization]: 
				self.text = func()
		
		logger.info('Text preprocessing is done!')

		return self.text 

<>:188: SyntaxWarning: invalid escape sequence '\d'
<>:188: SyntaxWarning: invalid escape sequence '\d'
C:\Users\vadig\AppData\Local\Temp\ipykernel_24108\3550293487.py:188: SyntaxWarning: invalid escape sequence '\d'
  '''


In [22]:
class BagOfWords: 
	"""
	Преобразует текст в мешок слов. 

	Мешок слов (Bag of Words) - это представление текста, которое описывает 
  частоту появления слов в документе. Это простая модель, игнорирующая 
  порядок слов, но сохраняющая их количество.  
	"""
	
	def __init__(self, preprocessed_text): 
		"""
		Инициализация класса. 

		:param preprocessed_text: предобработанный текст 
		"""

		logger.info('BagOfWords class is initialized')

		if isinstance(preprocessed_text, dict):
				logger.warning('Input is already a dictionary, not a text string')
				self.preprocessed_text = ' '.join(preprocessed_text.keys())
		else:
				self.preprocessed_text = preprocessed_text
	
	def create_bag(self): 
		"""
    Создает мешок слов с помощью sklearn.
    
    :return: словарь, где ключи - слова, значения - частота их появления
    """
		
		logger.info('Creating bag of words using sklearn...')
		
		try: 
			vectorizer = CountVectorizer()

			# fit() — обучает векторизатор 
			# transform() — преобразует текст в числовое представление
			# преобразует текст в матрицу: 
			# 1. строки соответствуют документа (в моем случае один)
			# 2. столбцы соответствуют уникальным словам из словаря 
			# 3. значения в ячейках — частота появления слов в соответствующем документе 
			#
			# X — объект scipy.sparse.csr_matrix (разреженная матрица)
			# CSR — compressed sparse row 
			# такая матрица используется для эффективного хранения данных, где большинство элементов 
			# равны нулю. 
			#
			# Пример матрицы: 
			# (0, 1)	1
			# (0, 3)	2
			# (0, 4)	1
			# (0, 7)	1
			# (0, 10)	1
			#
			# (строка, столбец) сколько раз встречается 
			X = vectorizer.fit_transform([self.preprocessed_text])
			words = vectorizer.get_feature_names_out()

			print(words)

			word_freq = {} 
			for i, word in enumerate(words): 
				count = X.toarray()[0][i]
				if count > 0: 
					word_freq[word] = count 

			logger.info(f'Bag of words created with {len(word_freq)} unique words using sklearn')
			return word_freq 
		
		except Exception as e: 
			logger.info(f'Bag of words created with {len(word_freq)} unique words using sklearn')
			return {} 

In [23]:
class NGrams: 
	"""
	Преобразует текст в N-граммы - последовательности из N слов.
	
	N-грамма - это последовательность из N элементов (в данном случае, слов)
	идущих друг за другом в тексте.
	"""

	def __init__(self, preprocessed_text):
		"""
		Инициализация класса.
		
		:param preprocessed_text: предобработанный текст
		"""

		logger.info('NGrams class is initialized')

		self.preprocessed_text = preprocessed_text

	def create_ngrams(self, n=2):
		"""
		Создаёт N-граммы из предобработанного текста. 

		:param n: размер N-граммы (количество слов в одной N-грамме)
    :return: словарь, где ключи - N-граммы, значения - частота их появления
		"""

		logger.info(f'Creating {n}-grams...')

		try: 
			words = self.preprocessed_text.split() 

			# Если текст короче, чем размер N-граммы, возвращаем пустой словарь
			if len(words) < n:
					logger.warning(f'Text is too short to create {n}-grams')
					return {}
      
			ngrams = []
			for i in range(len(words) - n + 1): 
				ngram = ' '.join(words[i:i+n])
				ngrams.append(ngram)

			ngram_freq = {} 
			for ngram in ngrams: 
				if ngram in ngram_freq: 
					ngram_freq[ngram] += 1 
				else: 
					ngram_freq[ngram] = 1 
			
			logger.info(f'{n}-grams created with {len(ngram_freq)} unique {n}-grams')
			return ngram_freq 
	
		except Exception as e: 
			logger.error(f'Error during {n}-grams creation: {str(e)}')
			return {} 


In [24]:
russian_input_file_path = 'D:/Projects/SvetlanaDmitrievna/lab3_ii/texts/russian_text.txt'
text = read_txt(russian_input_file_path)

output = TextPreprocessing(
	text, 
	delete_punctuation_mode='string', 
	text_normalization_method='stem').process_text()

2025-03-27 12:53:44,094 - INFO - Reading text from file: D:/Projects/SvetlanaDmitrievna/lab3_ii/texts/russian_text.txt
2025-03-27 12:53:44,111 - INFO - Successfully read 2111 chars from file.
2025-03-27 12:53:44,113 - INFO - Text is initializing...
2025-03-27 12:53:44,132 - INFO - Detected language: russian
2025-03-27 12:53:44,134 - INFO - Text is initialized with parameters: delete_punctuation_mode=string, text_normalization_method=stem
2025-03-27 12:53:44,135 - INFO - Text has been converted to lower case.
2025-03-27 12:53:44,138 - INFO - The text for step — Start. To lower case. — is written to D:/Projects/SvetlanaDmitrievna/lab3_ii/texts/output_russian.txt
2025-03-27 12:53:44,139 - INFO - Correction of typos initiated...
2025-03-27 12:53:45,792 - INFO - Corrected 0 typos using Yandex.Speller
2025-03-27 12:53:45,795 - INFO - The text for step — Spell correction. — is written to D:/Projects/SvetlanaDmitrievna/lab3_ii/texts/output_russian.txt
2025-03-27 12:53:45,796 - INFO - Convertin

In [25]:
# Записываем текст в файл 

text_to_txt(russian_output_file_path, output, new_text=False, step='Final Preproccessed Text')

2025-03-27 12:53:45,855 - INFO - Transmitted preprocessed text with 1524 chars.
2025-03-27 12:53:45,858 - INFO - The text for step — Final Preproccessed Text — is written to D:/Projects/SvetlanaDmitrievna/lab3_ii/texts/output_russian.txt


In [26]:
bag_of_words = BagOfWords(output).create_bag()
bigrams = NGrams(output).create_ngrams(n=2)
ngrams = NGrams(output).create_ngrams(n=4)


print(bag_of_words)
print(bigrams)
print(ngrams)

2025-03-27 12:53:45,874 - INFO - BagOfWords class is initialized
2025-03-27 12:53:45,876 - INFO - Creating bag of words using sklearn...
2025-03-27 12:53:45,883 - INFO - Bag of words created with 180 unique words using sklearn
2025-03-27 12:53:45,884 - INFO - NGrams class is initialized
2025-03-27 12:53:45,885 - INFO - Creating 2-grams...
2025-03-27 12:53:45,887 - INFO - 2-grams created with 229 unique 2-grams
2025-03-27 12:53:45,888 - INFO - NGrams class is initialized
2025-03-27 12:53:45,890 - INFO - Creating 4-grams...
2025-03-27 12:53:45,892 - INFO - 4-grams created with 237 unique 4-grams


['автобус' 'аренд' 'будильник' 'бутерброд' 'быстр' 'вернул' 'вечер' 'взял'
 'взят' 'вообщ' 'восемьпятьдес' 'восемьтридца' 'врем' 'встал' 'встат'
 'встрет' 'выключ' 'вып' 'вышел' 'гдет' 'год' 'город' 'гот' 'двадца'
 'двасорок' 'две' 'двест' 'девян' 'девятнадцатьнол' 'девятьсорок' 'деньг'
 'десят' 'добра' 'договор' 'дом' 'домик' 'друз' 'единиц' 'еха' 'ждал'
 'завтр' 'заказ' 'заказа' 'законч' 'замет' 'запыха' 'зарплат' 'зуб'
 'йогурт' 'кажд' 'каф' 'километр' 'клиент' 'коллег' 'компан' 'котор' 'коф'
 'куп' 'кусок' 'кухн' 'лег' 'лишн' 'месяц' 'минут' 'молок' 'нажима'
 'напитк' 'например' 'нача' 'начальник' 'нем' 'немн' 'неплох' 'нов' 'нол'
 'нольпятнадца' 'нужн' 'обед' 'общ' 'обычн' 'одобр' 'омлет' 'опаздыва'
 'опозда' 'оста' 'остановк' 'отлож' 'отмет' 'отчет' 'офис' 'пересла'
 'пешк' 'письм' 'пицц' 'планирова' 'плох' 'подготов' 'пойт' 'полчас'
 'поровн' 'последн' 'посмотрет' 'потрат' 'почемут' 'почист' 'почт' 'правд'
 'предлож' 'предостав' 'приготов' 'придет' 'примерн' 'принес' 'пришел'
 '

In [27]:
english_input_file_path = 'D:/Projects/SvetlanaDmitrievna/lab3_ii/texts/english_text.txt'
text = read_txt(english_input_file_path)

eng_output = TextPreprocessing(
	text, 
	text_normalization_method='lemma', 
	delete_punctuation_mode='string').process_text()

2025-03-27 12:53:45,909 - INFO - Reading text from file: D:/Projects/SvetlanaDmitrievna/lab3_ii/texts/english_text.txt
2025-03-27 12:53:45,912 - INFO - Successfully read 2217 chars from file.
2025-03-27 12:53:45,913 - INFO - Text is initializing...
2025-03-27 12:53:45,928 - INFO - Detected language: english
2025-03-27 12:53:45,929 - INFO - Text is initialized with parameters: delete_punctuation_mode=string, text_normalization_method=lemma
2025-03-27 12:53:45,931 - INFO - Text has been converted to lower case.
2025-03-27 12:53:45,933 - INFO - The text for step — Start. To lower case. — is written to D:/Projects/SvetlanaDmitrievna/lab3_ii/texts/output_english.txt
2025-03-27 12:53:45,934 - INFO - Correction of typos initiated...
2025-03-27 12:53:55,792 - INFO - Correction of typos initiated done with 441 corrected words.
2025-03-27 12:53:55,794 - INFO - The text for step — Spell correction. — is written to D:/Projects/SvetlanaDmitrievna/lab3_ii/texts/output_english.txt
2025-03-27 12:53:55

In [28]:
# Записываем текст в файл 

text_to_txt(english_output_file_path, eng_output, new_text=False, step='Final Preproccessed Text')

2025-03-27 12:53:55,863 - INFO - Transmitted preprocessed text with 1408 chars.
2025-03-27 12:53:55,866 - INFO - The text for step — Final Preproccessed Text — is written to D:/Projects/SvetlanaDmitrievna/lab3_ii/texts/output_english.txt


In [29]:
bag_of_words = BagOfWords(eng_output).create_bag()
bigrams = NGrams(eng_output).create_ngrams(n=2)
ngrams = NGrams(eng_output).create_ngrams(n=4)

print(bag_of_words)
print(bigrams)
print(ngrams)

2025-03-27 12:53:55,891 - INFO - BagOfWords class is initialized
2025-03-27 12:53:55,892 - INFO - Creating bag of words using sklearn...
2025-03-27 12:53:55,898 - INFO - Bag of words created with 174 unique words using sklearn
2025-03-27 12:53:55,899 - INFO - NGrams class is initialized
2025-03-27 12:53:55,900 - INFO - Creating 2-grams...
2025-03-27 12:53:55,902 - INFO - 2-grams created with 230 unique 2-grams
2025-03-27 12:53:55,903 - INFO - NGrams class is initialized
2025-03-27 12:53:55,905 - INFO - Creating 4-grams...
2025-03-27 12:53:55,906 - INFO - 4-grams created with 231 unique 4-grams


['accidentally' 'add' 'alarm' 'almost' 'arent' 'article' 'ask' 'asleep'
 'bad' 'battery' 'bed' 'bit' 'bos' 'bread' 'browsing' 'burn' 'buy' 'cant'
 'cash' 'cashier' 'cent' 'central' 'change' 'charger' 'checkout' 'chicken'
 'come' 'computer' 'cook' 'cooky' 'couldnt' 'date' 'day' 'decide' 'dinner'
 'disappear' 'dont' 'early' 'eighteen' 'eightthirty' 'elevenfortyfive'
 'end' 'enough' 'episode' 'everything' 'expiration' 'fall' 'fast'
 'favorite' 'fifteen' 'file' 'finally' 'five' 'forget' 'forgot'
 'forgotten' 'forty' 'found' 'four' 'fridge' 'friend' 'gallon' 'get' 'go'
 'good' 'hadnt' 'head' 'heat' 'help' 'hmm' 'home' 'hope' 'ill' 'inflation'
 'instead' 'keep' 'kept' 'later' 'liberty' 'like' 'loaf' 'long' 'maybe'
 'medium' 'meet' 'meeting' 'might' 'mike' 'milk' 'minute' 'much' 'need'
 'next' 'nine' 'oh' 'oil' 'old' 'onion' 'open' 'overall' 'oversleep'
 'pack' 'pan' 'park' 'phone' 'picked' 'pm' 'point' 'potato' 'pour' 'price'
 'probably' 'ran' 'reach' 'read' 'ready' 'realize' 'regret' 'relax